**Name: Adit Samir Patel**  
ASUID: 1229560217
Homework: 3
Task: 1

In [1]:
import math
import random
import time
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('data.csv', header = None).to_numpy()
data.shape

(10000, 784)

In [3]:
label = pd.read_csv('label.csv', header = None).to_numpy().flatten()
label.shape

(10000,)

**Task 3.1**

In [4]:
#calculating k value
k = len(set(list(label)))
print("Total Cluster:", k)

Total Cluster: 10


In [5]:
#Different distance metrics
def euclidean_distane(point1, point2):
    square = np.sum((point1-point2) ** 2)
    return np.sqrt(square)
    
    
def cosine_distance(point1, point2):
    dot_product = np.sum(point1*point2)
    sq_1 = np.sqrt(np.sum(point1 ** 2))
    sq_2 = np.sqrt(np.sum(point2 ** 2))
    
    cosine_similarity = dot_product/(sq_1 * sq_2)
    return 1 - cosine_similarity

def jaccard_distance(point1, point2):
    mini = np.sum(np.minimum(point1,point2))
    maxi = np.sum(np.maximum(point1,point2))
    
    jaccard_similarity = mini/maxi
    return 1 -jaccard_similarity

In [6]:
def which_distance(distance):
    if distance == "euclidean":
        return euclidean_distane
    elif distance == "cosine":
        return cosine_distance
    else:
        return jaccard_distance

# function to calculate Sum Squared Error
def calc_sse(clusters, new_centroids):
    sse = 0
    for i in range(len(clusters)):
        for j in range(len(clusters[i])):
            distance = np.sum((clusters[i][j] - new_centroids[i]) ** 2)
            sse += distance
    
    return sse     

In [7]:
def k_means_task1(data, k, distance):
    max_iter = 30
    n_samples = data.shape[0]
    total_sample = data.shape[0]
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    
    distance_metric = which_distance(distance)
    
    for i in range(max_iter):
        clusters = [[] for  _ in range(k)]
        for j in range(total_sample):
            cur_distance = [distance_metric(data[j],centroid) for centroid in centroids]
            cluster_index = np.argmin(cur_distance)
            clusters[cluster_index].append(data[j])
            
        
        new_centroids = [0]*k
        for i,cluster in enumerate(clusters):
            new_centroids[i] = np.mean(cluster, axis=0)
        
        new_centroids = np.array(new_centroids)
        
    sse = calc_sse(clusters, new_centroids)
    return new_centroids, sse   
            

In [8]:
distance = ["euclidean", "cosine", "jaccardian"]
for i in distance:
    centroids, sse = k_means_task1(data, k, i)
    print(f" Distance {i} : SSE : {sse}")

 Distance euclidean : SSE : 27737747750.16371
 Distance cosine : SSE : 28216802474.30625
 Distance jaccardian : SSE : 29138789677.54363


**Task 3.2**

In [9]:
def assign_cluster(cluster_indices):
    cmap = np.zeros(10000, dtype=int)   # Mapping clusters for each data point
    for i, indices in enumerate(cluster_indices):
        for index in indices:
            cmap[index] = i  

    return cmap

def calculate_accuracy(data, true_labels, cluster_indices, distance_metric):
    cluster_assignments = assign_cluster(cluster_indices)

    cluster_label_map = {}
    
    for cluster_idx, cluster_indices in enumerate(cluster_indices):
        labels_in_cluster = true_labels[cluster_indices]
    
        unique_labels, counts = np.unique(labels_in_cluster, return_counts=True)
        majority_label = unique_labels[np.argmax(counts)]
        
        cluster_label_map[cluster_idx] = majority_label

    mapped_labels = list(map(cluster_label_map.get, cluster_assignments))
    predicted_labels = np.array(mapped_labels)
    
    return np.mean(predicted_labels == true_labels)

In [10]:
def k_means_task2(data, label, k, distance):
    max_iter = 30
    n_samples = data.shape[0]
    total_sample = data.shape[0]
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    
    distance_metric = which_distance(distance)
    
    for i in range(max_iter):
        clusters = [[] for  _ in range(k)]
        cluster_indices =  [[] for  _ in range(k)]
        for j in range(total_sample):
            cur_distance = [distance_metric(data[j],centroid) for centroid in centroids]
            cluster_index = np.argmin(cur_distance)
            clusters[cluster_index].append(data[j])
            cluster_indices[cluster_index].append(j)
            
        
        new_centroids = [0]*k
        
        for i,cluster in enumerate(clusters):
            new_centroids[i] = np.mean(cluster, axis=0)
        
        new_centroids = np.array(new_centroids)
    
    accuracy = calculate_accuracy(data, label, cluster_indices, distance_metric)
    return accuracy

In [11]:
distance = ["euclidean", "cosine", "jaccardian"]
for i in distance:
    accuracy = k_means_task2(data, label, k, i)
    print(f" Distance: {i} : Accuracy:{accuracy}")

 Distance: euclidean : Accuracy:0.3929
 Distance: cosine : Accuracy:0.4397
 Distance: jaccardian : Accuracy:0.444


**Task 3.3**

In [12]:
# Function to count iterations. stopping Criteria when next SSE greater than prev SSE
def k_means_task3_stop_criteria_sse(data, k, distance):
    max_iter = 300
    n_samples = data.shape[0]
    total_sample = data.shape[0]
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    
    distance_metric = which_distance(distance)
    
    prev_sse = math.inf
    iterations = 0
    while iterations < max_iter:
        clusters = [[] for  _ in range(k)]
        iterations += 1
        for j in range(total_sample):
            cur_distance = [distance_metric(data[j],centroid) for centroid in centroids]
            cluster_index = np.argmin(cur_distance)
            clusters[cluster_index].append(data[j])
            
        
        new_centroids = [0]*k
        
        for i,cluster in enumerate(clusters):
            new_centroids[i] = np.mean(cluster, axis=0)
        
        new_centroids = np.array(new_centroids)
        cur_sse = calc_sse(clusters, new_centroids)
        
        print(f" Iterations {iterations} sse {cur_sse}")
        if prev_sse == math.inf:
            prev_sse = cur_sse
        elif cur_sse > prev_sse:
            return cur_sse, iterations
    return cur_sse, iterations
       

In [13]:
distance = ["euclidean", "cosine", "jaccardian"]
for i in distance:
    start_time = time.time()
    sse, iterations = k_means_task3_stop_criteria_sse(data, k, i)
    end_time = time.time()
    print(f" Distance: {i} : SSE : {sse}  Iterations: {iterations} Time Taken to Converge {end_time - start_time}")

 Iterations 1 sse 28192691144.00199
 Iterations 2 sse 28192691144.00199
 Iterations 3 sse 28192691144.00199
 Iterations 4 sse 28192691144.00199
 Iterations 5 sse 28192691144.00199
 Iterations 6 sse 28192691144.00199
 Iterations 7 sse 28192691144.00199
 Iterations 8 sse 28192691144.00199
 Iterations 9 sse 28192691144.00199
 Iterations 10 sse 28192691144.00199
 Iterations 11 sse 28192691144.00199
 Iterations 12 sse 28192691144.00199
 Iterations 13 sse 28192691144.00199
 Iterations 14 sse 28192691144.00199
 Iterations 15 sse 28192691144.00199
 Iterations 16 sse 28192691144.00199
 Iterations 17 sse 28192691144.00199
 Iterations 18 sse 28192691144.00199
 Iterations 19 sse 28192691144.00199
 Iterations 20 sse 28192691144.00199
 Iterations 21 sse 28192691144.00199
 Iterations 22 sse 28192691144.00199
 Iterations 23 sse 28192691144.00199
 Iterations 24 sse 28192691144.00199
 Iterations 25 sse 28192691144.00199
 Iterations 26 sse 28192691144.00199
 Iterations 27 sse 28192691144.00199
 Iteration

In [14]:
#stopping Criteria when change is centroid is very small
def k_means_task3_stop_criteria_centroid(data, k, distance):
    max_iter = 30
    n_samples = data.shape[0]
    total_sample = data.shape[0]
    centroids = data[np.random.choice(n_samples, k, replace=False)]
    
    distance_metric = which_distance(distance)
    
    iterations = 0
    while iterations < 300:
        clusters = [[] for  _ in range(k)]
        iterations += 1
        for j in range(total_sample):
            cur_distance = [distance_metric(data[j],centroid) for centroid in centroids]
            cluster_index = np.argmin(cur_distance)
            clusters[cluster_index].append(data[j])
            
        new_centroids = [0]*k
        
        for i,cluster in enumerate(clusters):
            new_centroids[i] = np.mean(cluster, axis=0)
        
        new_centroids = np.array(new_centroids)
        
        if np.all(np.abs(new_centroids - centroids) < 0.0001):
            break
        else:
            centroids = new_centroids
            
    cur_sse = calc_sse(clusters, new_centroids)
    return iterations,cur_sse

In [15]:
distance = ["euclidean", "cosine", "jaccardian"]
for i in distance:
    start_time = time.time()
    iterations, sse = k_means_task3_stop_criteria_centroid(data, k, i)
    end_time = time.time()
    print(f" Distance: {i} : Iterations: {iterations} SSE: {sse} Time Taken to converge {end_time - start_time}")

 Distance: euclidean : Iterations: 51 SSE: 25596431631.762955 Time Taken to converge 58.57218956947327
 Distance: cosine : Iterations: 102 SSE: 25556844562.94031 Time Taken to converge 240.3712728023529
 Distance: jaccardian : Iterations: 136 SSE: 25416834184.548878 Time Taken to converge 217.7992067337036
